In [5]:
import numpy as np
import neuroglancer as ng


## Set the neuroglancer client to be one that is hosted locally


In [2]:
# ng.set_static_content_source(url='http://localhost:8080')
ng.set_static_content_source(url='https://neuroglancer-braincogs.appspot.com')
# ng.set_static_content_source(url='https://alpine-dogfish-285314.appspot.com')
# ng.set_static_content_source(url='https://neuromancer-seung-import.appspot.com')
# ng.set_static_content_source(url='https://neuroglancer-demo.appspot.com')


# Load in the raw data into neuroglancer and generate the link

## Host the raw-space allen atlas cloudvolume

You will need to host this with cloudvolume as was done for the raw data above. Use a different port though, e.g.
```python
import cloudvolume
vol = cloudvolume.Cloudvolume('file:///jukebox/LightSheetTransfer/kelly/neuroglancer/201908_cfos/m61468_observ_rawatlas')
vol.viewer(port=1338) # must be a different port than the raw data
```
Again, make sure to do this outside of this jupyter notebook

## Load in the atlas
Load in a segmentation layer (e.g. the raw-space allen atlas) that is being hosted with cloudvolume using the same viewer object that you already made. If you go back to the neuroglancer window after running this it should be there in a new layer

In [115]:
viewer = ng.Viewer()
with viewer.txn() as s:
#     s.layers['seg'] = ng.SegmentationLayer(source='precomputed://gs://wanglab-pma/allenatlas_2017_16bit_hierarch_labels_fillmissing/'
#     )
    s.layers['cfos'] = ng.ImageLayer(source='precomputed://http://localhost:1337',
                                     shader="""void main() {emitGrayscale(1.0-toNormalized(getDataValue())*25.0);}""")
print(viewer)

http://127.0.0.1:40741/v/886bc92e23fc9084a963964b94f060ed148b254c/


In [116]:
# Create a 1mm x 1mm x 1mm  subvolume in the center of the brain in which to do the annotations
# This code colors the 
npix = 1 # we only need an empty cube with just a single voxel
nm_per_pix = 1000000 # each voxel is 1mm or 1e6 nm
# Decide where to put the subvolume in mm offset from the origin (the top, front, left of the brain, typically)
# May take some experimenting
mm_offset = [3,4,5] # x y z
# Don't change below
voxel_offset = [int(x*1e6/nm_per_pix) for x in mm_offset]
a = np.zeros((npix, npix, npix), dtype=np.uint8)

with viewer.txn() as s:
    s.layers['training_set_overlay'] = ng.ImageLayer(
        source=ng.LocalVolume(
            a,
            dimensions=ng.CoordinateSpace(
                scales=[nm_per_pix, nm_per_pix, nm_per_pix], # 10 micron isotropic
                units=['nm', 'nm', 'nm'],
                names=['x', 'y', 'z']),
            voxel_offset=voxel_offset), # This 
        shader="""
void main() {
  emitRGB(vec3(toNormalized(getDataValue(0)),
               toNormalized(getDataValue(1)),
               toNormalized(getDataValue(2))));
}
""",
    )
    # Then pan to the location of this new volume
    # To do that, need dimensions of your original data layer
    dim_dict = s.dimensions
    nm_scale_dict = {}
    for coord in ['x','y','z']:
        dim_obj = dim_dict[coord]
        unit = dim_obj.unit
        assert unit == 'm'
        nm_scale_dict[coord] = dim_obj.scale*1e9
    # find the position of the new subvolume in these units
    x_subvol_orig = mm_offset[0]*1e6/nm_scale_dict['x']
    y_subvol_orig = mm_offset[1]*1e6/nm_scale_dict['y']
    z_subvol_orig = mm_offset[2]*1e6/nm_scale_dict['z']
    # Pan/zoom to this location
    s.voxel_coordinates = [x_subvol_orig,y_subvol_orig,z_subvol_orig]
    s.cross_section_scale=0.5
    

You should see a square show up in at least one of the 2D panels and in the 3d viewer panel. Right click it in any of the panels to pan to it. 

In [119]:
with viewer.txn() as s:
    # Swap layers to make subvolume pop out
    s.layers[1],s.layers[0] = s.layers[0],s.layers[1]
    s.layers[1].opacity=0.6

This should make the subvolume look something like this against the background image:
<img src="screenshots/subvolume_popout.png">

This should make it easier to keep track of whether you are in the subvolume while annotating. Adjust the opacity of the original data layer as well as the image contrast ("d" and "f" keys) to get the best contrast between the subvolume and the background image. You can turn the bounding box of the subvolume off with the "shift+b" keystroke.

In [120]:
# Make an annotation layer so you can start making annotations
with viewer.txn() as s:
    s.layers['subvol_annotation']=ng.AnnotationLayer()
    annot_layer = s.layers['subvol_annotation']
    annot_layer.tool = "annotatePoint"
    s.cross_section_depth=-80 # the z-depth in microns over which the annotated points will persist. negative sign is necessary for some reason
    s.selected_layer.layer = 'subvol_annotation' 
    s.selected_layer.visible = True
with viewer.config_state.txn() as st:
    st.status_messages['update'] = 'Start annotating points by ctrl+left clicking'   
    

In [125]:
with viewer.txn() as s:
    print(s)

ViewerState({"dimensions": {"x": [5e-06, "m"], "y": [5e-06, "m"], "z": [1e-05, "m"]}, "position": [638.9795532226562, 865.268798828125, 506.5], "crossSectionScale": 0.1019628058671067, "crossSectionDepth": -0.15625, "projectionScale": 4096, "layers": [{"type": "image", "source": "python://volume/886bc92e23fc9084a963964b94f060ed148b254c.9c9d903498b482deade4924fbbebad642d72a398", "opacity": 1, "shader": "void main() {emitGrayscale(1.0-toNormalized(getDataValue())*25.0);}", "name": "training_set_overlay"}, {"type": "image", "source": "precomputed://http://localhost:1337", "opacity": 0.6, "shader": "void main() {emitGrayscale(1.0-toNormalized(getDataValue())*25.0);}", "name": "cfos"}, {"type": "annotation", "source": {"url": "local://annotations", "transform": {"outputDimensions": {"x": [5e-06, "m"], "y": [5e-06, "m"], "z": [1e-05, "m"]}}}, "tool": "annotatePoint", "annotations": [{"point": [638.1748046875, 844.96044921875, 505.5], "type": "point", "id": "8cc043a2eb64d7b40bcae0f0c56f68a791

In [126]:
with viewer.txn() as s:
    s.cross_section_depth=-80

In [ ]:
with viewer.txn() as s:
